In [48]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [49]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [50]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [51]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df

In [52]:
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
data_df = review_df.withColumn("star_rating", review_df["star_rating"].cast(IntegerType()))
vine_df = data_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)




+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [53]:
twenty_plus_votes_df = vine_df.filter("total_votes >= 20")
twenty_plus_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [54]:
votes_calc_df = twenty_plus_votes_df.withColumn('useful_votes',(twenty_plus_votes_df["helpful_votes"] / twenty_plus_votes_df["total_votes"]*100))
votes_calc_df.filter('useful_votes >= 50')
votes_calc_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     useful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|91.52542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|71.42857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|95.23809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|63.63636363636363|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|95.74468085106383|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
only showing top 5 rows



In [55]:
paid_prog_df = votes_calc_df.filter(votes_calc_df['vine'] == 'Y')
paid_count= paid_prog_df.count()

In [56]:
unpaid_prog_df = votes_calc_df.filter(votes_calc_df['vine']=='N')
unpaid_count = unpaid_prog_df.count()

In [57]:
five_star_paid_df = paid_prog_df.filter('star_rating = 5')
five_star_paid_count = five_star_paid_df.count()

In [58]:
five_star_unpaid_df = unpaid_prog_df.filter('star_rating = 5')
five_star_unpaid_count = five_star_unpaid_df.count()

In [59]:
percentage_five_star_paid = (five_star_paid_count / paid_count)*100
print ("The percentage of 5 star paid reviews was :%.2f" % percentage_five_star_paid)

The percentage of 5 star paid reviews was :36.08


In [60]:
percentage_five_star_unpaid = (five_star_unpaid_count / unpaid_count)*100
print ("The percentage of unpaid 5 star reviews was :%.2f" % percentage_five_star_unpaid)

The percentage of unpaid 5 star reviews was :43.62
